# AI Tutor   

1. Install and Import Gradio
2. Create .env variable and add OpenAI sk
3. Import OpenAI API code snips
4. Load the SS question files as dataframe
    - Output of dataframe 
5. Response functionality should refer to questions in the SS question file
    - The bot should be able to read the file in the 
6. Save responses or csv in txt and use as context for sessions future
7. Prompts engineering should be used to build a very very very good Python Data Science Tutor
    - Preload the tutor wi/Datascienceth Python docu (future feature...use store memory and context in vector database?)mentation
    - Preload the tutor with Data science documentation
    - Tutor should be able to create a game
8. Chatbot buttons
    - Test me : uses database levels 1, 2, 3
    - Track my progress: chatbot checks answer submission and judges if you got answer right or wrong
    - Python flash cards built-in functionality


In [4]:
# Install and Import Gradio, Pandas, Altair, Matplotlib, Os, and OpenAI (put this in "bot_libs.py" file and import it in one line as "import bot_libs" )
import gradio as gr
import pandas as pd
import os
import openai
from pathlib import Path #read in txt files for clean bot 
from dotenv import load_dotenv



In [5]:
# # Get a list of files and directories in the specified path...used during troubleshooting

# os.getcwd()
# os.chdir('C:\\Users\\matte\\OneDrive\\Desktop\\Lonely Octopus\\Freelance Project\\Chatbot\\version 1')

In [6]:
# Load Question DF
question_df = pd.read_csv('question_df.csv')

In [11]:
# Tool bot is designed to be the tool-using feature of the chat bot. 
# During the session it will be a second instance of GPT-3.5 that can use tools
# This gives pretty consistent behavior
with open('tool bot sys content.txt', 'r') as file:
    tool_bot_sys_content = file.read()

def tool_bot(prompt, history=[]):
    messages =[
        {"role": "system", "content": tool_bot_sys_content}
    ]
       
    messages.append({'role': 'user', 'content': f'Here is my question or statement: {prompt}'}) # using f string to avoid prompt injections
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0, #play with temp to get more factual responses
        max_tokens=10,#further limit output
        top_p=0,
        frequency_penalty=0,
        presence_penalty=0
    )


    # saving the message we want to show to the user
    assistant_content = response['choices'][0]['message']['content']

    # Visual check for testing purposes
    print(response) 

    return assistant_content
   
# #testing tool_bot

# prompt = input()
# tool_bot(prompt=prompt) 

In [12]:
#html parser bot
# bot should take url, grab certain data from the website
# should also be able to see if you failed the question and see what you submitted
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from bs4 import BeautifulSoup
import time

# Function that takes url as input
def html_scraper(url):

    # Path to the GeckoDriver executable
    GECKODRIVER_PATH = 'geckodriver.exe'
    
    # Initialize the WebDriver
    options = webdriver.FirefoxOptions()
    service = FirefoxService(executable_path=GECKODRIVER_PATH)
    driver = webdriver.Firefox(service=service, options=options)
    
    # Open a web page
    # url = input('url')
    driver.get(url)
    
    # Wait for JavaScript to load content (adjust time as needed)
    time.sleep(5)  # Wait for 5 seconds
    
    # Get HTML content
    html_content = driver.page_source
    
    # # Parse HTML content with Beautiful Soup...used to get all tags and explore data
    # soup = BeautifulSoup(html_content, 'html.parser')
    # for tag in soup.find_all(True):
    #     print(f"Tag: {tag.name}")
    #     print(f"Attributes: {tag.attrs}")
    #     content = str(tag.contents).strip()
    #     print(f"Content: {content[:100]}...") if len(content) > 100 else print(f"Content: {content}")
    #     print("-" * 50) # Separator line
    # ... (other code)
    
    # Get HTML content
    html_content = driver.page_source
    
    # Parse HTML content with Beautiful Soup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the first h1 tag
    h1_tag = soup.find('h1')
    
    # Print the content if the h1 tag exists
    if h1_tag:
        print("Content of the h1 tag:", h1_tag.text)
    else:
        print("No h1 tag found on the page.")
    
    # Close the browser
    driver.quit()
    return h1_tag.text

url = input('url')
question_short = html_scraper(url)
print(question_short)



url https://platform.stratascratch.com/coding/10354-most-profitable-companies?code_type=2


C:\Users\matte\AppData\Local\Temp\ipykernel_12560\3023275108.py:17: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  service = FirefoxService(executable_path=GECKODRIVER_PATH)


Content of the h1 tag: Most Profitable Companies
Most Profitable Companies


In [13]:
# Pulling out the question row
question_row = question_df.loc[question_df['question_short'] == question_short]
# print(question_row)

# convert DataFrame to numpy array to list in order to extract the string objects contained inside
hint = question_row['hint'].values.tolist()
py_hint = question_row['python_hint'].values.tolist()
question = question_row['question'].values.tolist()
py_solution = question_row['solution'].values.tolist()

print(f'''{question[0]}

{hint[0]}

{py_hint[0]}

{py_solution[0]}
''')

-- Find the 3 most profitable companies in the entire world.
 -- Output the result along with the corresponding company name.
 -- Sort the result based on profits in descending order.

- Use **rank()** function to ensure an edge cases.

- Use sort_values(column_name, order) to sort along a specified column; Set order to False to display the printed values in descending order 
 - Limit rows to be printed by specifying rank on profits.
 - Use [ [ column_name/s] ] to return a specified column of the dataframe

SELECT company,
  profit
 FROM
  (SELECT *,
  rank() OVER (
  ORDER BY profit DESC) as rank
  FROM
  (SELECT company,
  sum(profits) AS profit
  FROM forbes_global_2010_2014
  GROUP BY company) sq) sq2
 WHERE rank <=3



In [14]:
# FAQ HTML file Writer Bot

def faq_html(xlsx, question_short):
    df = pd.read_excel(xlsx)
    df = df[df['question'] == question_short]
    html_table = df.to_html(index=False)
    html_content = f'''<!DOCTYPE html><html><head><title>FAQs for {question_short}</title></head><body>'''
    
    html_content += html_table
    html_content += '</body></html>'
    
    with open(f'''{question_short} faq.html''', 'w') as file:
        file.write(html_content)

faq_html(xlsx='ai_tutor_faqs.xlsx',question_short='Most Profitable Companies')


In [40]:
# The AI Tutor Backend. It has a nested tool_bot function
# Which will trigger grabbing data from questions database
with open('ai tutor sys content.txt', 'r') as file:
    ai_tutor_sys_content = file.read()


# # we store the 0 index with an placeholder '' string so that we don't get index error for index being out of range. 

# hint = ['']
# py_hint = ['']
# question = ['']
# py_solution = ['']

# Global messages list. The memory of the conversation is stored in this list
messages =[{"role": "system","content": ai_tutor_sys_content + f'''
Here is the context to consider as you help the student:
Question: 
{question[0]}

Hint: 
{hint[0]}

Python Hint (code snippet):
{py_hint[0]}

Solution: 
{py_solution[0]} '''}]

#track index of conversation:
i = [0]

load_dotenv()
def ai_chat(prompt, history):
    # calling messages variable inside the function
    global messages
            
    tool_response = tool_bot(prompt) #this is another instance of OpenAI (basically another agent) to issue commands
 
    # tool_dict = {}
    # for tool in tool_dict:
    #     if tool_response == tool:
    #         assistant_content = tool_dict[tool]
    #         return assistant_content
            
    openai.api_key = os.getenv("OPENAI_API_KEY") ##This isn't working for some reason 
    messages.append({'role': 'user', 'content': f'{prompt}'}) #f string to avoid prompt injection errors from user
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=1, #play with temp to get more factual responses
        max_tokens=500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )


    assistant_content = response['choices'][0]['message']['content']
    # hint = '' #this is parsing the json file that is the response reponse. you take the first item in choices list (a list of dictionaries), go to message key, and then go to content key 
    
    #print this in the CLI or notebook. This helps with understanding how the response json is structured for further customization
    print(f'''
    
    MESSAGE: {messages}
    
    RESPONSE:{response}
    
    ''')
    
    # Saving as variable to store the conversation history
    format_str = f'''
    User: {prompt}
    
    Response: {assistant_content}
    
    '''

    # Saving content to a text file
    # Open a file in write mode, 'a' argument serves as appending to
    with open('saved_chat.txt', 'a') as file:
        file.write(format_str)

    #add token counter and a way to handle hitting token limit.  
    # Prob summarizing previous assistant/student info into 2-4 sentences, 3000 tokens is safe place to start summary
    token_count = response['usage']['total_tokens']
    if token_count >= 3000:
        # Reload message history text for summarization. Better formatted than parsing the response JSON
        # content_txt = Path('saved_chat.txt').read_text() 
        # summary_str[0] = summarize_bot_saved(content_txt)
        messages = [messages[0]] # basically erase the memory except the system message and kept it as a list
        # print(f'This is the `messages` memory: {summary_str}')
    
    messages.append(({'role':'assistant', 'content': assistant_content}))
    absolute_path = os.path.abspath('matt/Most Profitable Companies faq.html')
    print(absolute_path)

    if i[0] == 0:
        # assistant_content = f'''<a href="{absolute_path}" target="_blank">Click here</a>'''
        assistant_content = "Hi there! I am your AI Tutor. Please click the FAQ button to download a list of FAQs for this particular question"
        i[0] += 1
   
    
    return assistant_content

demo = gr.ChatInterface(
    fn=ai_chat, 
    submit_btn = 'Submit', 
    retry_btn = 'Retry',
    clear_btn = 'Clear',
    title = 'AI Tutor V1',
    #add a save convo button that can create html file that can then be printed like a pdf later
    )

demo.launch()

Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.


{
  "id": "chatcmpl-7ozVKmOFBSZsHwO2ruJnVbfzLjcgG",
  "object": "chat.completion",
  "created": 1692387178,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "xxxxx"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 273,
    "completion_tokens": 2,
    "total_tokens": 275
  }
}

    
    MESSAGE: [{'role': 'system', 'content': "You are an data science tutor with 20+ years of experience but you also follow instructions very well. You are speaking to a student who has 0-5 years of experience in the data science field. Here are some instructions for you:\nALWAYS ALWAYS ALWAYS format your response as instructed. Do not deviate under any circumstances.\n\n1. In this set of instructions, everything in between ``` and ``` characters are the exact format you should ALWAYS ALWAYS ALWAYS use. Text between '<' and '>' characters are descriptions of what type of response to in

In [ ]:
''' simplify database, only save the question_short and the solution. Bot then parses html of current page user is on in 
order to look up what question and the solution are as context, and dynamically generates hints and walkthrough. 
This can be saved to a csv for future reference as well. THIS FEATURE WOULD NEED TO BE BUILT ON THE STATASCRATCH SIDE. IN INTEREST OF TIME
WE COULD ONLY DEMO WHAT IT WOULD LOOK LIKE WHILE NAVAGATING A PAGE ON THE WEBSITE. ALSO LOOK TO INTEGRATE THE SCROLL DOWN MENU TO SELECT WHICH QUESTION YOURE ON

Copy paste the url and bot will scrape the data everytime the user prompts the bot to get the current state. 
INSTEAD GO FOR USING THE CONTEXT APPROACH, APP SHOULD BE USED TO GIVE WALKTHROUGH, EDGE CASE, AND GENERAL CODING ASSISTANCE,
USING THE CURRENT QUESTION AS CONTEXT. when you click AI tutor button, it should know which question you are on. 

The pseudocode on the backend should look something like
if button pressed:
    check user's current url
    select row in database the corresponds to that question url
    feed in data from that row as context for the tutor
    when "Walkthrough" or "Explain Solution" button is pressed it takes the database as context and uses LLM pretraining to give walkthrough or explain the solution
    

can also save all conversations that can be analyzed for future referece by the team at statascratch. Can use an LLM to
then categorize types of questions asked and perform analytics.'''

In [ ]:
# Add functionality that tracks tokens and keeps system message, but deletes or summarizes conversation into a few lines, and feeds resets token count back to where it was at beginning of convo.
#FAQ database:

#Save responses, create an openai instance that can read each entry column one at atime (to avoid token limit)
# and categorize commonly asked responses...and save that as a csv for future analysis

#Future functionality:
#Human like responsiveness. randomized delayed timer
#Voice optionality. 
